In [1]:
import pandas as pd
import numpy as np

import warnings

from select import select

warnings.filterwarnings('ignore')

In [2]:
path = "E:\\MIMIC_III\\mimic-iii-clinical-database-1.4"

In [3]:
# load Patients and Diagnoses_icd
patients = pd.read_csv(path + '\\patients.csv')
diagnoses = pd.read_csv(path + '\\diagnoses_icd.csv')

In [4]:
print(f"Patients shape is {patients.shape}")
print(f"Diagnoses shape is {diagnoses.shape}")
print()

# Check if ICD codes are numeric or not
icd_column_dtype = diagnoses['ICD9_CODE'].dtype
print("Checking ICD codes:")
print(f"\tICD codes column dtype is {icd_column_dtype}")

# Check if all ICD codes are numeric
if pd.api.types.is_numeric_dtype(diagnoses['ICD9_CODE']):
    print("\tAll ICD codes are numeric.")
else:
    print("\tICD codes are not entirely numeric. Attempting to clean and convert...")

    # Attempt to clean and convert to numeric, ignoring errors
    diagnoses['ICD9_CODE'] = pd.to_numeric(diagnoses['ICD9_CODE'], errors='coerce')

    # Report how many could not be converted
    null_count = diagnoses['ICD9_CODE'].isnull().sum()
    print(f"\tConverted ICD codes to numeric. {null_count} rows could not be converted and contain NaN.")


Patients shape is (46520, 8)
Diagnoses shape is (651047, 5)

Checking ICD codes:
	ICD codes column dtype is object
	ICD codes are not entirely numeric. Attempting to clean and convert...
	Converted ICD codes to numeric. 97296 rows could not be converted and contain NaN.


In [5]:
# checking why ICD9_CODE columns is Object
diagnoses['ICD9_CODE'].isna().sum()

97296

In [6]:
diagnoses_cleaned = diagnoses.dropna(subset=['ICD9_CODE'])
# Verify the result
print(f"Diagnoses shape after dropping missing ICD9_CODE: {diagnoses_cleaned.shape}")

Diagnoses shape after dropping missing ICD9_CODE: (553751, 5)


In [7]:
# Check if ICD codes are numeric or not
icd_column_dtype = diagnoses_cleaned['ICD9_CODE'].dtype
print("Checking ICD codes:")
print(f"\tICD codes column dtype is {icd_column_dtype}")

# Check if all ICD codes are numeric
if pd.api.types.is_numeric_dtype(diagnoses['ICD9_CODE']):
    print("\tAll ICD codes are numeric.")
else:
    print("\tICD codes are not entirely numeric. Attempting to clean and convert...")

    # Attempt to clean and convert to numeric, ignoring errors
    diagnoses['ICD9_CODE'] = pd.to_numeric(diagnoses['ICD9_CODE'], errors='coerce')

    # Report how many could not be converted
    null_count = diagnoses['ICD9_CODE'].isnull().sum()
    print(f"\tConverted ICD codes to numeric. {null_count} rows could not be converted and contain NaN.")


Checking ICD codes:
	ICD codes column dtype is float64
	All ICD codes are numeric.


In [8]:
# now lets extract sepsis patients ( sepsis , severe sepsis , septic shock)
sepsis_codes = {
    99591.0: 'Sepsis',
    99592.0: 'Severe Sepsis',
    78552.0: 'Septic Shock'
}

In [9]:
# Filter for sepsis-related diagnoses
sepsis_diagnoses = diagnoses_cleaned[diagnoses_cleaned['ICD9_CODE'].isin(sepsis_codes.keys())]


In [10]:
sepsis_diagnoses.columns

Index(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'SEQ_NUM', 'ICD9_CODE'], dtype='object')

In [11]:
sepsis_diagnoses.shape

(7770, 5)

In [12]:
# Merge with the patients table to get demographic details
sepsis_patients = sepsis_diagnoses.merge(patients, on='SUBJECT_ID', how='inner')

In [13]:
print(sepsis_patients.head())
sepsis_patients.shape

   ROW_ID_x  SUBJECT_ID  HADM_ID  SEQ_NUM  ICD9_CODE  ROW_ID_y GENDER  \
0      1547         117   164853     16.0    99592.0       108      F   
1      1604         124   138376      6.0    99592.0       114      M   
2       505          64   172056      3.0    99591.0        57      F   
3       679          85   112077     18.0    99591.0        77      M   
4       131          21   111970      2.0    78552.0        18      M   

                   DOB                  DOD             DOD_HOSP  \
0  2083-12-28 00:00:00  2133-12-01 00:00:00  2133-12-01 00:00:00   
1  2090-11-19 00:00:00  2166-02-01 00:00:00  2166-02-01 00:00:00   
2  2116-06-27 00:00:00                  NaN                  NaN   
3  2090-09-18 00:00:00  2167-09-12 00:00:00  2167-09-12 00:00:00   
4  2047-04-04 00:00:00  2135-02-08 00:00:00  2135-02-08 00:00:00   

               DOD_SSN  EXPIRE_FLAG  
0  2133-12-01 00:00:00            1  
1  2166-02-01 00:00:00            1  
2                  NaN            0  


(7770, 12)

In [14]:
print(len(diagnoses['ICD9_CODE'].unique()))

5954


## ----------------------------------------------------

## ----------------------------------------------------

# Now exclude patients with multiple admissions except for first admissions

In [15]:
admissions = pd.read_csv(path + '\\admissions.csv')

In [16]:
admissions.shape

(58976, 19)

In [17]:
admissions.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,BRAIN MASS,0,1
3,24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,NaN,NaN,INTERIOR MYOCARDIAL INFARCTION,0,1
4,25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1


In [18]:
admissions['ADMITTIME'].isna().sum()

0

In [19]:
# Count the number of admissions per patient
# finding admission count isn't useful because we need to delete multiple sepsis admission except first
admission_counts = admissions.groupby('SUBJECT_ID').size().reset_index(name='ADMISSION_COUNT')

print(admission_counts)


       SUBJECT_ID  ADMISSION_COUNT
0               2                1
1               3                1
2               4                1
3               5                1
4               6                1
...           ...              ...
46515       99985                1
46516       99991                1
46517       99992                1
46518       99995                1
46519       99999                1

[46520 rows x 2 columns]


In [20]:
# Filter out patients with multiple admissions
# as we see here is not useful
single_admissions_patients = admission_counts[admission_counts['ADMISSION_COUNT'] == 1]

print(single_admissions_patients)

       SUBJECT_ID  ADMISSION_COUNT
0               2                1
1               3                1
2               4                1
3               5                1
4               6                1
...           ...              ...
46515       99985                1
46516       99991                1
46517       99992                1
46518       99995                1
46519       99999                1

[38983 rows x 2 columns]


In [21]:
# choose sepsis patients from admissions
filtered_admissions = admissions[admissions['SUBJECT_ID'].isin(sepsis_patients['SUBJECT_ID'])]
filtered_admissions.shape

(8703, 19)

In [22]:
filtered_admissions.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
19,40,38,185910,2166-08-10 00:28:00,2166-09-04 11:30:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,LONG TERM CARE HOSPITAL,Medicare,NaN,CATHOLIC,WIDOWED,WHITE,NaN,NaN,ACUTE MYOCARDIAL INFARCTION-SEPSIS,0,1
22,456,357,174486,2197-12-06 07:13:00,2198-01-03 14:00:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Private,ENGL,NOT SPECIFIED,MARRIED,WHITE,2197-12-06 06:28:00,2197-12-06 07:43:00,GASTROINTESTINAL BLEED,0,1
23,457,357,145674,2198-08-02 04:49:00,2198-10-26 14:55:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Private,ENGL,NOT SPECIFIED,MARRIED,WHITE,2198-08-02 02:03:00,2198-08-02 05:37:00,SEIZURE,0,1
24,458,357,122609,2198-11-01 22:36:00,2198-11-14 14:20:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Private,ENGL,NOT SPECIFIED,MARRIED,WHITE,2198-11-01 18:01:00,2198-11-01 23:06:00,SEPSIS,0,1
25,459,357,101651,2199-10-20 12:05:00,2199-10-23 17:30:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Private,ENGL,NOT SPECIFIED,MARRIED,WHITE,2199-10-20 09:34:00,2199-10-20 13:25:00,PNEUMONIA,0,1


In [23]:
filtered_admissions = filtered_admissions[['SUBJECT_ID', 'ADMITTIME']].dropna(subset=['ADMITTIME'])
filtered_admissions.head()

,SUBJECT_ID,ADMITTIME
19,38,2166-08-10 00:28:00
22,357,2197-12-06 07:13:00
23,357,2198-08-02 04:49:00
24,357,2198-11-01 22:36:00
25,357,2199-10-20 12:05:00


In [24]:
filtered_admissions = filtered_admissions.sort_values(by='ADMITTIME')
filtered_admissions.head()

,SUBJECT_ID,ADMITTIME
7378,4521,2100-06-28 19:29:00
37595,31585,2100-07-02 19:28:00
38886,42357,2100-07-14 02:04:00
50353,73131,2100-07-14 18:14:00
54560,84585,2100-08-03 00:26:00


In [56]:
print(sepsis_patients["SUBJECT_ID"].unique().__len__())
print(sepsis_patients.shape)

4689
(7770, 12)


In [55]:
print(filtered_admissions["SUBJECT_ID"].unique().__len__())
print(filtered_admissions.shape)

4689
(4689, 2)


In [58]:
# Sort the DataFrame by ADMITTIME
filtered_admissions_sorted = filtered_admissions.sort_values(by='ADMITTIME')

# Group by `subject_id` and keep the first occurrence for duplicates
unique_or_first_admissions = filtered_admissions_sorted.groupby('SUBJECT_ID', as_index=False).first()

unique_or_first_admissions.shape

(4689, 2)

In [59]:
unique_or_first_admissions.head()

,SUBJECT_ID,ADMITTIME
0,21,2134-09-11 12:17:00
1,38,2166-08-10 00:28:00
2,61,2118-06-17 14:40:00
3,62,2113-02-15 00:19:00
4,64,2143-03-03 09:25:00


In [71]:
print(unique_or_first_admissions.SUBJECT_ID.duplicated().sum())

0


In [72]:
sepsis_patients_corrected = unique_or_first_admissions

# ----------------------------------------------

# Now we should implement filters

In [29]:
patients.DOB.sample(10)

10417   2180-01-12
23226   2091-03-05
1140    2137-12-16
21758   2036-06-18
2385    2085-03-22
6083    2089-07-12
13585   2116-12-01
16934   2032-04-13
44826   2077-09-05
30377   2152-03-31
Name: DOB, dtype: datetime64[ns]

In [29]:
# Convert DOB to datetime format
patients['DOB'] = pd.to_datetime(patients['DOB'], errors='coerce')

# Identify shifted DOBs (year < 1900)
shifted_dobs = patients[patients['DOB'].dt.year < 1900]

# Count the number of shifted DOBs
shifted_count = shifted_dobs.shape[0]

# Display the result
print(f"Number of shifted DOB entries: {shifted_count}")

# Optionally, view the shifted rows
print(shifted_dobs[['SUBJECT_ID', 'DOB']].head())

Number of shifted DOB entries: 1959
     SUBJECT_ID        DOB
52          698 1864-11-16
61          709 1817-01-20
161         813 1879-12-15
174         827 1874-01-04
237          19 1808-08-05


In [30]:
shifted_dobs.head()

,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
52,658,698,F,1864-11-16,2168-04-22 00:00:00,NaN,2168-04-22 00:00:00,1
61,667,709,F,1817-01-20,2118-10-13 00:00:00,NaN,2118-10-13 00:00:00,1
161,767,813,M,1879-12-15,2180-01-30 00:00:00,2180-01-30 00:00:00,2180-01-30 00:00:00,1
174,780,827,F,1874-01-04,NaN,NaN,NaN,0
237,16,19,M,1808-08-05,2109-08-18 00:00:00,NaN,2109-08-18 00:00:00,1


In [30]:
# Convert DOB to datetime format
patients['DOB'] = pd.to_datetime(patients['DOB'], errors='coerce')

# Drop rows with shifted DOBs (year < 1900)
patients_cleaned = patients[patients['DOB'].dt.year >= 1900]

# Display the number of remaining rows
print(f"Number of rows after dropping shifted DOBs: {patients_cleaned.shape[0]}")


Number of rows after dropping shifted DOBs: 44561


In [35]:
patients_cleaned.shape

(44561, 8)

In [31]:
patients_cleaned['DOB'] = patients_cleaned['DOB'].dt.to_pydatetime()
admissions['ADMITTIME'] = admissions['ADMITTIME'].dt.to_pydatetime()

In [38]:
filtered_admissions["ADMITTIME"] = filtered_admissions["ADMITTIME"].dt.to_pydatetime()

AttributeError: Can only use .dt accessor with datetimelike values

In [40]:
filtered_admissions["ADMITTIME"].head()

7378     2100-06-28 19:29:00
37595    2100-07-02 19:28:00
38886    2100-07-14 02:04:00
50353    2100-07-14 18:14:00
54560    2100-08-03 00:26:00
Name: ADMITTIME, dtype: object

In [33]:
# Merge PATIENTS (cleaned) and ADMISSIONS (cleaned)
merged_df = pd.merge(
    patients_cleaned[['SUBJECT_ID', 'DOB']],
    admissions[['SUBJECT_ID', 'ADMITTIME']],
    on='SUBJECT_ID',
    how='inner'
)

# Calculate age using native Python datetime
merged_df['AGE'] = merged_df.apply(
    lambda row: (row['ADMITTIME'].to_pydatetime() - row['DOB'].to_pydatetime()).days // 365 if row['DOB'] and row['ADMITTIME'] else None,
    axis=1
)

# Display results
print(merged_df[['SUBJECT_ID', 'DOB', 'ADMITTIME', 'AGE']].head())

   SUBJECT_ID        DOB           ADMITTIME  AGE
0         249 2075-03-13 2149-12-17 20:41:00   74
1         249 2075-03-13 2155-02-03 20:16:00   79
2         249 2075-03-13 2156-04-27 15:33:00   81
3         250 2164-12-27 2188-11-12 09:22:00   23
4         251 2090-03-15 2110-07-27 06:46:00   20


In [34]:
merged_df.shape

(56401, 4)

In [37]:
sepsis_patients['DOB'] = sepsis_patients['DOB'].dt.to_pydatetime()
# Filter rows where the year in DOB is 1872
dob_1872 = sepsis_patients[sepsis_patients['DOB'].dt.year == 1872]

# Display the filtered rows
print(dob_1872)

# Optionally, count the rows with DOB in 1872
print(f"Number of rows with DOB in 1872: {dob_1872.shape[0]}")


      ROW_ID_x  SUBJECT_ID  HADM_ID  SEQ_NUM  ICD9_CODE  ROW_ID_y GENDER  \
697      41071        3706   179428     12.0    78552.0      3503      F   
698      41072        3706   179428     13.0    99592.0      3503      F   
5391    434574       52164   129810      3.0    99592.0     34007      F   
7410    620611       93454   142925      7.0    99592.0     44808      F   
7411    620612       93454   142925      8.0    78552.0     44808      F   

            DOB                  DOD             DOD_HOSP  \
697  1872-12-18  2173-02-13 00:00:00                  NaN   
698  1872-12-18  2173-02-13 00:00:00                  NaN   
5391 1872-11-25  2172-12-02 00:00:00                  NaN   
7410 1872-07-31  2172-08-01 00:00:00  2172-08-01 00:00:00   
7411 1872-07-31  2172-08-01 00:00:00  2172-08-01 00:00:00   

                  DOD_SSN  EXPIRE_FLAG  
697   2173-02-13 00:00:00            1  
698   2173-02-13 00:00:00            1  
5391  2172-12-02 00:00:00            1  
7410        

In [38]:
sepsis_patients.DOB.sample(10)

3141   2083-10-19
2676   2060-09-05
1608   2111-06-19
1805   2127-04-05
7165   2022-11-12
6266   2049-12-10
2140   2080-06-10
5839   1884-03-22
7145   2055-04-18
1821   2071-02-04
Name: DOB, dtype: datetime64[ns]

In [39]:
# Save the sepsis_patients DataFrame as a CSV file
sepsis_patients.to_csv("sepsis_patients.csv", index=False)

print("sepsis_patients has been saved as 'sepsis_patients.csv'.")


sepsis_patients has been saved as 'sepsis_patients.csv'.


In [40]:
merged_df.shape

(16267, 4)

In [41]:
merged_df.head()

,SUBJECT_ID,DOB,ADMITTIME,AGE
0,117,2083-12-28,2133-04-07 16:29:00,49
1,117,2083-12-28,2133-11-13 21:34:00,49
2,124,2090-11-19,2161-12-17 03:39:00,71
3,124,2090-11-19,2165-05-21 21:02:00,74
4,124,2090-11-19,2165-12-31 18:55:00,75


In [49]:
sepsis_patients.shape


(7770, 12)

In [53]:
# Check duplicates in SUBJECT_ID in sepsis_patients
print("Duplicates in sepsis_patients:")
print(sepsis_patients['SUBJECT_ID'].duplicated().sum())

# Check duplicates in SUBJECT_ID in merged_df
print("Duplicates in merged_df:")
print(merged_df['SUBJECT_ID'].duplicated().sum())

# Find SUBJECT_ID counts in both DataFrames
print("Value counts in sepsis_patients:")
print(sepsis_patients['SUBJECT_ID'].value_counts().head())

print("Value counts in merged_df:")
print(merged_df['SUBJECT_ID'].value_counts().head())


Duplicates in sepsis_patients:
3081
Duplicates in merged_df:
11927
Value counts in sepsis_patients:
SUBJECT_ID
41976    18
3977     11
4787     10
73713    10
29426    10
Name: count, dtype: int64
Value counts in merged_df:
SUBJECT_ID
41976    270
73713    170
4787     160
11318    133
3977     132
Name: count, dtype: int64


In [52]:
# Merge to add 'AGE' column from merged_df to sepsis_patients
merged_final_df = pd.merge(
    sepsis_patients,
    merged_df[['SUBJECT_ID', 'AGE']],  # Only 'SUBJECT_ID' and 'AGE' from merged_df
    on='SUBJECT_ID',                   # Merge on SUBJECT_ID
    how='inner'                        # Keep only matching SUBJECT_ID rows
)

# Print the resulting shape and first rows
print("Shape of the new merged DataFrame:", merged_final_df.shape)
print(merged_final_df.head())

# Save to a new CSV file
merged_final_df.to_csv('sepsis_patients_with_age.csv', index=False)
print("Merged data with 'AGE' column saved to 'sepsis_patients_with_age.csv'")

Shape of the new merged DataFrame: (52077, 13)
   ROW_ID_x  SUBJECT_ID  HADM_ID  SEQ_NUM  ICD9_CODE  ROW_ID_y GENDER  \
0      1547         117   164853     16.0    99592.0       108      F   
1      1547         117   164853     16.0    99592.0       108      F   
2      1604         124   138376      6.0    99592.0       114      M   
3      1604         124   138376      6.0    99592.0       114      M   
4      1604         124   138376      6.0    99592.0       114      M   

         DOB                  DOD             DOD_HOSP              DOD_SSN  \
0 2083-12-28  2133-12-01 00:00:00  2133-12-01 00:00:00  2133-12-01 00:00:00   
1 2083-12-28  2133-12-01 00:00:00  2133-12-01 00:00:00  2133-12-01 00:00:00   
2 2090-11-19  2166-02-01 00:00:00  2166-02-01 00:00:00  2166-02-01 00:00:00   
3 2090-11-19  2166-02-01 00:00:00  2166-02-01 00:00:00  2166-02-01 00:00:00   
4 2090-11-19  2166-02-01 00:00:00  2166-02-01 00:00:00  2166-02-01 00:00:00   

   EXPIRE_FLAG  AGE  
0            1   

In [56]:
# Drop duplicates in merged_df based on SUBJECT_ID
merged_df_unique = merged_df.drop_duplicates(subset=['SUBJECT_ID'])

# Merge with sepsis_patients
merged_final_df = pd.merge(
    sepsis_patients,
    merged_df_unique[['SUBJECT_ID', 'AGE']],  # Use unique SUBJECT_ID from merged_df
    on='SUBJECT_ID',
    how='inner'
)

# Print the new shape
print("Shape of merged DataFrame after removing duplicates:", merged_final_df.shape)
merged_final_df.shape




Shape of merged DataFrame after removing duplicates: (7179, 13)


(7179, 13)

In [57]:
merged_final_df.head()

,ROW_ID_x,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE,ROW_ID_y,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG,AGE
0,1547,117,164853,16.0,99592.0,108,F,2083-12-28,2133-12-01 00:00:00,2133-12-01 00:00:00,2133-12-01 00:00:00,1,49
1,1604,124,138376,6.0,99592.0,114,M,2090-11-19,2166-02-01 00:00:00,2166-02-01 00:00:00,2166-02-01 00:00:00,1,71
2,505,64,172056,3.0,99591.0,57,F,2116-06-27,NaN,NaN,NaN,0,26
3,679,85,112077,18.0,99591.0,77,M,2090-09-18,2167-09-12 00:00:00,2167-09-12 00:00:00,2167-09-12 00:00:00,1,71
4,131,21,111970,2.0,78552.0,18,M,2047-04-04,2135-02-08 00:00:00,2135-02-08 00:00:00,2135-02-08 00:00:00,1,87


In [59]:
filtered_sepsis_patientes_under18_df = merged_final_df[merged_final_df['AGE'] > 18]

# Print the shape and first few rows of the filtered DataFrame
print("Shape after removing under 18 ages:", filtered_sepsis_patientes_under18_df.shape)
filtered_sepsis_patientes_under18_df.head()

Shape after removing under 18 ages: (7161, 13)


,ROW_ID_x,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE,ROW_ID_y,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG,AGE
0,1547,117,164853,16.0,99592.0,108,F,2083-12-28,2133-12-01 00:00:00,2133-12-01 00:00:00,2133-12-01 00:00:00,1,49
1,1604,124,138376,6.0,99592.0,114,M,2090-11-19,2166-02-01 00:00:00,2166-02-01 00:00:00,2166-02-01 00:00:00,1,71
2,505,64,172056,3.0,99591.0,57,F,2116-06-27,NaN,NaN,NaN,0,26
3,679,85,112077,18.0,99591.0,77,M,2090-09-18,2167-09-12 00:00:00,2167-09-12 00:00:00,2167-09-12 00:00:00,1,71
4,131,21,111970,2.0,78552.0,18,M,2047-04-04,2135-02-08 00:00:00,2135-02-08 00:00:00,2135-02-08 00:00:00,1,87


In [62]:
# Load the required MIMIC-III files for demographic data
patients_df = pd.read_csv(path + '\\patients.csv', usecols=['SUBJECT_ID'])
admissions_df = pd.read_csv(path + '\\admissions.csv', usecols=['SUBJECT_ID', 'ETHNICITY'])
# Merge demographic data from patients and admissions on SUBJECT_ID
demographic_df = pd.merge(
    patients_df,
    admissions_df.drop_duplicates(subset=['SUBJECT_ID']),
    on='SUBJECT_ID',
    how='inner'
)
print(demographic_df.head())

   SUBJECT_ID               ETHNICITY
0         249                   WHITE
1         250  BLACK/AFRICAN AMERICAN
2         251   UNKNOWN/NOT SPECIFIED
3         252                   WHITE
4         253                   WHITE


In [64]:
# Merge the demographic data with the filtered DataFrame (adults only)
final_df_with_demo = pd.merge(
    filtered_sepsis_patientes_under18_df,           # Filtered data without under-18 patients
    demographic_df,        # Demographic data
    on='SUBJECT_ID',
    how='left'             # Left join to retain all rows in filtered_df
)
final_df_with_demo.head()

,ROW_ID_x,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE,ROW_ID_y,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG,AGE,ETHNICITY
0,1547,117,164853,16.0,99592.0,108,F,2083-12-28,2133-12-01 00:00:00,2133-12-01 00:00:00,2133-12-01 00:00:00,1,49,WHITE
1,1604,124,138376,6.0,99592.0,114,M,2090-11-19,2166-02-01 00:00:00,2166-02-01 00:00:00,2166-02-01 00:00:00,1,71,WHITE
2,505,64,172056,3.0,99591.0,57,F,2116-06-27,NaN,NaN,NaN,0,26,BLACK/AFRICAN AMERICAN
3,679,85,112077,18.0,99591.0,77,M,2090-09-18,2167-09-12 00:00:00,2167-09-12 00:00:00,2167-09-12 00:00:00,1,71,WHITE
4,131,21,111970,2.0,78552.0,18,M,2047-04-04,2135-02-08 00:00:00,2135-02-08 00:00:00,2135-02-08 00:00:00,1,87,WHITE


In [66]:
final_cleaned_df = final_df_with_demo.dropna(subset=['GENDER', 'DOB', 'ETHNICITY'])

# Print the resulting shape and a preview
print("Shape after removing patients without demographic data:", final_cleaned_df.shape)
final_cleaned_df.head()

Shape after removing patients without demographic data: (7161, 14)


,ROW_ID_x,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE,ROW_ID_y,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG,AGE,ETHNICITY
0,1547,117,164853,16.0,99592.0,108,F,2083-12-28,2133-12-01 00:00:00,2133-12-01 00:00:00,2133-12-01 00:00:00,1,49,WHITE
1,1604,124,138376,6.0,99592.0,114,M,2090-11-19,2166-02-01 00:00:00,2166-02-01 00:00:00,2166-02-01 00:00:00,1,71,WHITE
2,505,64,172056,3.0,99591.0,57,F,2116-06-27,NaN,NaN,NaN,0,26,BLACK/AFRICAN AMERICAN
3,679,85,112077,18.0,99591.0,77,M,2090-09-18,2167-09-12 00:00:00,2167-09-12 00:00:00,2167-09-12 00:00:00,1,71,WHITE
4,131,21,111970,2.0,78552.0,18,M,2047-04-04,2135-02-08 00:00:00,2135-02-08 00:00:00,2135-02-08 00:00:00,1,87,WHITE


In [67]:
# Load lab events and lab items tables
labevents = pd.read_csv(path + '\\labevents.csv', usecols=['SUBJECT_ID', 'ITEMID', 'CHARTTIME', 'VALUENUM', 'VALUEUOM', 'FLAG'])
labitems = pd.read_csv(path + '\\d_labitems.csv', usecols=['ITEMID', 'LABEL', 'CATEGORY', 'FLUID'])

# Merge labevents with labitems to get descriptive labels for lab tests
lab_results = pd.merge(
    labevents,
    labitems,
    on='ITEMID',
    how='left'
)

In [74]:
print("Shape of lab_results:", lab_results.shape)
lab_results.head()

Shape of lab_results: (27854055, 9)


,SUBJECT_ID,ITEMID,CHARTTIME,VALUENUM,VALUEUOM,FLAG,LABEL,FLUID,CATEGORY
0,3,50820,2101-10-12 16:07:00,7.39,units,NaN,pH,Blood,Blood Gas
1,3,50800,2101-10-12 18:17:00,NaN,NaN,NaN,SPECIMEN TYPE,BLOOD,BLOOD GAS
2,3,50802,2101-10-12 18:17:00,-1.00,mEq/L,NaN,Base Excess,Blood,Blood Gas
3,3,50804,2101-10-12 18:17:00,22.00,mEq/L,NaN,Calculated Total CO2,Blood,Blood Gas
4,3,50808,2101-10-12 18:17:00,0.93,mmol/L,abnormal,Free Calcium,Blood,Blood Gas


In [75]:
sepsis_with_lab_pd = pd.merge(
    final_cleaned_df,
    lab_results,
    on = 'SUBJECT_ID',
    how = 'left'
) 
sepsis_with_lab_pd.head()

,ROW_ID_x,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE,ROW_ID_y,GENDER,DOB,DOD,DOD_HOSP,...,AGE,ETHNICITY,ITEMID,CHARTTIME,VALUENUM,VALUEUOM,FLAG,LABEL,FLUID,CATEGORY
0,1547,117,164853,16.0,99592.0,108,F,2083-12-28,2133-12-01 00:00:00,2133-12-01 00:00:00,...,49,WHITE,51277.0,2133-11-21 02:51:00,18.90,%,abnormal,RDW,Blood,Hematology
1,1547,117,164853,16.0,99592.0,108,F,2083-12-28,2133-12-01 00:00:00,2133-12-01 00:00:00,...,49,WHITE,51279.0,2133-11-21 02:51:00,3.22,m/uL,abnormal,Red Blood Cells,Blood,Hematology
2,1547,117,164853,16.0,99592.0,108,F,2083-12-28,2133-12-01 00:00:00,2133-12-01 00:00:00,...,49,WHITE,51301.0,2133-11-21 02:51:00,4.00,K/uL,NaN,White Blood Cells,Blood,Hematology
3,1547,117,164853,16.0,99592.0,108,F,2083-12-28,2133-12-01 00:00:00,2133-12-01 00:00:00,...,49,WHITE,50868.0,2133-11-21 04:51:00,20.00,mEq/L,NaN,Anion Gap,Blood,Chemistry
4,1547,117,164853,16.0,99592.0,108,F,2083-12-28,2133-12-01 00:00:00,2133-12-01 00:00:00,...,49,WHITE,50882.0,2133-11-21 04:51:00,22.00,mEq/L,NaN,Bicarbonate,Blood,Chemistry


In [76]:
sepsis_with_lab_pd.shape

(12888312, 22)

In [78]:
# Drop SUBJECT_IDs that do not appear in labevents
filtered_df_with_labs = final_cleaned_df[final_cleaned_df['SUBJECT_ID'].isin(lab_results['SUBJECT_ID'])]
labevents_unique = labevents.drop_duplicates(subset=['SUBJECT_ID'], keep='first')
final_with_labs = pd.merge(
    filtered_df_with_labs,
    labevents_unique[['SUBJECT_ID', 'ITEMID', 'CHARTTIME', 'VALUENUM', 'VALUEUOM', 'FLAG']],
    on='SUBJECT_ID',
    how='inner'  # Ensure we keep only matching SUBJECT_IDs
)
print("Shape after keeping unique lab results per SUBJECT_ID:", final_with_labs.shape)
final_with_labs.head()

Shape after keeping unique lab results per SUBJECT_ID: (7159, 19)


,ROW_ID_x,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE,ROW_ID_y,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG,AGE,ETHNICITY,ITEMID,CHARTTIME,VALUENUM,VALUEUOM,FLAG
0,1547,117,164853,16.0,99592.0,108,F,2083-12-28,2133-12-01 00:00:00,2133-12-01 00:00:00,2133-12-01 00:00:00,1,49,WHITE,51277,2133-11-21 02:51:00,18.90,%,abnormal
1,1604,124,138376,6.0,99592.0,114,M,2090-11-19,2166-02-01 00:00:00,2166-02-01 00:00:00,2166-02-01 00:00:00,1,71,WHITE,51466,2165-05-21 23:57:00,NaN,NaN,NaN
2,505,64,172056,3.0,99591.0,57,F,2116-06-27,NaN,NaN,NaN,0,26,BLACK/AFRICAN AMERICAN,51279,2143-06-23 19:11:00,4.32,m/uL,delta
3,679,85,112077,18.0,99591.0,77,M,2090-09-18,2167-09-12 00:00:00,2167-09-12 00:00:00,2167-09-12 00:00:00,1,71,WHITE,50903,2162-08-19 07:00:00,2.00,Ratio,NaN
4,131,21,111970,2.0,78552.0,18,M,2047-04-04,2135-02-08 00:00:00,2135-02-08 00:00:00,2135-02-08 00:00:00,1,87,WHITE,51274,2134-09-15 01:47:00,15.50,sec,abnormal


In [81]:
# Calculate the percentage of missing values for each row
missing_percentage = final_with_labs.isnull().mean(axis=1) * 100

# Filter out rows where the percentage of missing values exceeds 20%
final_cleaned_df_no_missing = final_with_labs[missing_percentage <= 20]

# Print the shape and a few rows of the cleaned DataFrame
print("Shape after removing rows with >20% missing data:", final_cleaned_df_no_missing.shape)
final_cleaned_df_no_missing.head()

Shape after removing rows with >20% missing data: (5381, 19)


,ROW_ID_x,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE,ROW_ID_y,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG,AGE,ETHNICITY,ITEMID,CHARTTIME,VALUENUM,VALUEUOM,FLAG
0,1547,117,164853,16.0,99592.0,108,F,2083-12-28,2133-12-01 00:00:00,2133-12-01 00:00:00,2133-12-01 00:00:00,1,49,WHITE,51277,2133-11-21 02:51:00,18.90,%,abnormal
1,1604,124,138376,6.0,99592.0,114,M,2090-11-19,2166-02-01 00:00:00,2166-02-01 00:00:00,2166-02-01 00:00:00,1,71,WHITE,51466,2165-05-21 23:57:00,NaN,NaN,NaN
2,505,64,172056,3.0,99591.0,57,F,2116-06-27,NaN,NaN,NaN,0,26,BLACK/AFRICAN AMERICAN,51279,2143-06-23 19:11:00,4.32,m/uL,delta
3,679,85,112077,18.0,99591.0,77,M,2090-09-18,2167-09-12 00:00:00,2167-09-12 00:00:00,2167-09-12 00:00:00,1,71,WHITE,50903,2162-08-19 07:00:00,2.00,Ratio,NaN
4,131,21,111970,2.0,78552.0,18,M,2047-04-04,2135-02-08 00:00:00,2135-02-08 00:00:00,2135-02-08 00:00:00,1,87,WHITE,51274,2134-09-15 01:47:00,15.50,sec,abnormal


In [82]:
chartevents = pd.read_csv(path + '\\chartevents.csv')

MemoryError: Unable to allocate 512. KiB for an array with shape (65536,) and data type int64

In [51]:
# Merge with sepsis_patients to add AGE
sepsis_with_age = pd.merge(
    sepsis_patients,
    merged_df[['SUBJECT_ID', 'AGE']],
    on='SUBJECT_ID',
    how='left'
)

sepsis_with_age.shape

(52668, 13)

In [ ]:
# Drop duplicates in merged_df based on SUBJECT_ID
merged_df_unique = merged_df.drop_duplicates(subset=['SUBJECT_ID'])

# Merge with sepsis_patients
merged_final_df = pd.merge(
    sepsis_patients,
    merged_df_unique[['SUBJECT_ID', 'AGE']],  # Use unique SUBJECT_ID from merged_df
    on='SUBJECT_ID',
    how='inner'
)

# Print the new shape
print("Shape of merged DataFrame after removing duplicates:", merged_final_df.shape)
merged_final_df.shape


In [44]:
# Filter merged_df to include only SUBJECT_IDs present in sepsis_patients
filtered_merged_df = merged_df[merged_df['SUBJECT_ID'].isin(sepsis_patients['SUBJECT_ID'])]

# Display the filtered DataFrame
print(filtered_merged_df)


       SUBJECT_ID        DOB           ADMITTIME  AGE
0             117 2083-12-28 2133-04-07 16:29:00   49
1             117 2083-12-28 2133-11-13 21:34:00   49
2             124 2090-11-19 2161-12-17 03:39:00   71
3             124 2090-11-19 2165-05-21 21:02:00   74
4             124 2090-11-19 2165-12-31 18:55:00   75
...           ...        ...                 ...  ...
16262       95561 2046-10-18 2114-05-22 14:54:00   67
16263       95561 2046-10-18 2114-06-07 17:51:00   67
16264       95803 2089-04-28 2140-08-21 00:24:00   51
16265       97143 2128-04-05 2189-03-07 22:14:00   60
16266       97158 2104-11-22 2165-08-03 01:36:00   60

[16267 rows x 4 columns]


In [45]:
merged_df.AGE.sample(10)

11451    45
1445     68
16263    67
12251    60
3864     27
8141     62
3664     44
7170     76
6776     66
8113     39
Name: AGE, dtype: int64

In [46]:
# Filter merged_df to include only patients under 18
under_18_df = merged_df[merged_df['AGE'] < 18]

# Display the filtered DataFrame
print(under_18_df)

# Count the number of patients under 18
print(f"Number of patients under 18: {under_18_df.shape[0]}")


      SUBJECT_ID        DOB           ADMITTIME  AGE
1219        1916 2156-01-09 2156-01-09 08:33:00    0
1251        4832 2154-03-01 2154-04-04 15:40:00    0
2475        8079 2141-12-02 2141-12-02 11:52:00    0
7743       25277 2171-11-01 2171-11-01 01:45:00    0
9615       30686 2150-08-04 2150-08-04 07:50:00    0
9932       32433 2201-01-11 2201-01-11 18:40:00    0
9933       32433 2201-01-11 2201-05-07 14:52:00    0
Number of patients under 18: 7


In [47]:
merged_df.shape

(16267, 4)